# import

In [10]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


# directories


In [11]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

# read exemple

In [16]:
dept = '1'
td007 =pd.read_csv(data_dir/'depts'/dept/'td007_paroi_opaque.csv',dtype=str)
td006 =pd.read_csv(data_dir/'depts'/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/'depts'/dept/'td001_dpe.csv',dtype=str)
td008 =pd.read_csv(data_dir/'depts'/dept/'td008_baie.csv',dtype=str)


In [18]:
td006.columns

Index(['id', 'td001_dpe_id', 'besoin_chauffage', 'deperdition_enveloppe',
       'deperdition_renouvellement_air', 'tv014_permeabilite_id',
       'tv015_valeur_conventionnelle_renouvellement_air_id',
       'tv026_classe_inertie_id', 'altitude', 'nombre_niveau', 'hsp_moyenne',
       'nombre_appartements', 'cclim', 'comclim'],
      dtype='object')

In [13]:
from td007_processing import merge_td007_tr_tv, postprocessing_td007
from td008_processing import merge_td008_tr_tv, postprocessing_td008
from td001_merge import merge_td001_dpe_id_envelope
from td007_processing import agg_td007_to_td001_essential, agg_surface_envelope
from td008_processing import agg_td008_to_td001_essential

td008_raw_cols = td008.columns
td007_raw_cols = td007.columns
td008 = merge_td008_tr_tv(td008)
td008 = postprocessing_td008(td008)

td007 = merge_td007_tr_tv(td007)
td007 = postprocessing_td007(td007, td008)

td001, td006, td007, td008 = merge_td001_dpe_id_envelope(td001=td001, td006=td006, td007=td007, td008=td008)

agg_td007 = agg_td007_to_td001_essential(td007)
agg_td008 = agg_td008_to_td001_essential(td008)
agg_surfaces = agg_surface_envelope(td007, td008)

agg = pd.concat([agg_td007, agg_td008, agg_surfaces], axis=1)

agg.index.name = 'td001_dpe_id'
cols = [el for el in td008.columns if el not in td008_raw_cols]
cols.append('td008_baie_id')
td008_p = td008[cols]
cols = [el for el in td007.columns if el not in td007_raw_cols]
cols.append('td007_paroi_opaque_id')
td007_p = td007[cols]

D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\utils.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[pond_col] = table[pond]
D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\utils.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[pond_value_col_temp] = table[pond_col] * table[value_col]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value 

ValueError: cannot reindex from a duplicate axis

In [39]:
agg.to_csv('td001_agg_enveloppe.csv')

In [14]:
agg

NameError: name 'agg' is not defined

In [44]:
agg.columns.tolist()

['Umurs_ext_avg',
 'Umurs_deper_avg',
 'Uplancher_bas_deper_avg',
 'Uplancher_haut_deper_avg',
 'is_plancher_bas_deper_isole',
 'is_plancher_haut_deper_isole',
 'is_murs_ext_isole',
 'is_murs_deper_isole',
 'mat_murs_deper_top',
 'mat_murs_ext_top',
 'mat_plancher_bas_deper_top',
 'mat_plancher_haut_deper_top',
 'Ubaie_avg',
 'type_vitrage_simple_top',
 'orientation_concat',
 'surf_murs_totale',
 'surf_murs_deper',
 'surf_murs_ext',
 'surf_pb_totale',
 'surf_pb_deper',
 'surf_ph_totale',
 'surf_ph_deper',
 'surface_vitree_totale',
 'surface_porte_totale',
 'surfaces_vitree_orientee_est',
 'surfaces_vitree_orientee_est_ou_ouest',
 'surfaces_vitree_orientee_horizontale',
 'surfaces_vitree_orientee_nondef',
 'surfaces_vitree_orientee_nord',
 'surfaces_vitree_orientee_ouest',
 'surfaces_vitree_orientee_sud',
 'ratio_surface_vitree']

In [21]:
td001.merge(agg.reset_index(),on='td001_dpe_id')

,td001_dpe_id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,...,surface_vitree_totale,surface_porte_totale,surfaces_vitree_orientee_est,surfaces_vitree_orientee_est_ou_ouest,surfaces_vitree_orientee_horizontale,surfaces_vitree_orientee_nondef,surfaces_vitree_orientee_nord,surfaces_vitree_orientee_ouest,surfaces_vitree_orientee_sud,ratio_surface_vitree
0,291,1338V1000003D,6988,3,1,3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-04,...,11.04,NaN,4.04,NaN,NaN,NaN,NaN,7.00,NaN,0.095950
1,493,1338V1000005F,6988,3,1,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-09,...,10.53,NaN,7.00,NaN,NaN,NaN,NaN,3.53,NaN,0.127374
2,708,1338V1000006G,6988,3,1,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-11,...,21.75,NaN,5.70,NaN,NaN,NaN,NaN,7.35,8.70,0.202326
3,1466,1338V1000007H,6988,3,1,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-19,...,23.50,NaN,7.60,NaN,NaN,NaN,NaN,6.20,9.70,0.225096
4,1708,1338V1000008I,6988,3,1,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-17,...,15.85,NaN,10.70,NaN,NaN,NaN,NaN,NaN,5.15,0.579525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81826,9430979,2038L1001773P,20347,3,9,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2020-03-10,...,25.26,NaN,8.00,NaN,NaN,NaN,5.6,0.49,11.17,0.262169
81827,9431119,2038L1001774Q,698,3,9,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2020-03-10,...,11.26,NaN,NaN,NaN,NaN,NaN,4.5,NaN,6.76,0.603107
81828,9431307,2038L1001775R,15341,3,9,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2020-03-10,...,8.81,NaN,8.81,NaN,NaN,NaN,NaN,NaN,NaN,0.520686
81829,9431405,2038L1001776S,15341,3,9,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2020-03-10,...,13.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.35,0.598655


In [62]:
louche = table.coefficient_transmission_thermique_paroi==0.35

In [90]:
table.loc[louche].epaisseur_isolation.value_counts()

10.0    40823
0.0      2117
8.0       144
9.0        13
4.0         8
15.0        6
5.0         6
13.0        5
6.0         4
12.0        3
11.0        2
20.0        1
Name: epaisseur_isolation, dtype: int64

In [93]:
1/0.35-1/2

2.357142857142857

In [95]:
2.35*4

9.4

In [89]:
table.loc[louche][['coefficient_transmission_thermique_paroi','tv003_Umur','epaisseur_isolation','resistance_thermique_isolation']]

,coefficient_transmission_thermique_paroi,tv003_Umur,epaisseur_isolation,resistance_thermique_isolation
0,0.35,NaN,10.0,0.0
1,0.35,NaN,10.0,0.0
18,0.35,2.5,10.0,0.0
19,0.35,2.5,10.0,0.0
20,0.35,2.5,10.0,0.0
...,...,...,...,...
439798,0.35,2.5,10.0,0.0
439800,0.35,2.5,10.0,0.0
439881,0.35,2.5,10.0,0.0
439889,0.35,2.5,10.0,0.0


In [99]:
table.loc[louche].epaisseur_isolation.value_counts()

10.0    40823
0.0      2117
8.0       144
9.0        13
4.0         8
15.0        6
5.0         6
13.0        5
6.0         4
12.0        3
11.0        2
20.0        1
Name: epaisseur_isolation, dtype: int64

In [102]:
table.loc[louche][['deperdition_thermique','surface_paroi','tv003_umur_id','epaisseur_isolation','epaisseur_isolation_calc','resistance_thermique_isolation']]

,deperdition_thermique,surface_paroi,tv003_umur_id,epaisseur_isolation,epaisseur_isolation_calc,resistance_thermique_isolation
0,40.27,115.06,NaN,10.0,NaN,0.0
1,3.59,10.80,NaN,10.0,NaN,0.0
18,9.51,27.35,TV003_001,10.0,NaN,0.0
19,2.94,8.80,TV003_001,10.0,NaN,0.0
20,4.64,13.90,TV003_001,10.0,NaN,0.0
...,...,...,...,...,...,...
439798,2.77,7.89,TV003_001,10.0,NaN,0.0
439800,7.19,20.46,TV003_001,10.0,NaN,0.0
439881,45.83,132.20,TV003_001,10.0,NaN,0.0
439889,23.44,66.96,TV003_001,10.0,NaN,0.0


In [73]:
table.

0         0.35
1         0.35
3         0.38
4         0.32
7         2.00
          ... 
439930    2.00
439931    2.00
439932    0.80
439934    0.40
439935    0.40
Name: coefficient_transmission_thermique_paroi, Length: 266952, dtype: float64

In [60]:
table.coefficient_transmission_thermique_paroi.value_counts()

2.00    46781
0.35    43135
0.36    13843
0.45    12000
0.40     8969
        ...  
1.58        1
0.01        1
1.91        1
1.61        1
1.89        1
Name: coefficient_transmission_thermique_paroi, Length: 210, dtype: int64

In [48]:
table

,td007_paroi_opaque_id,td006_batiment_id,tr014_type_parois_opaque_id,reference,deperdition_thermique,tv001_coefficient_reduction_deperditions_id,tv002_local_non_chauffe_id,coefficient_transmission_thermique_paroi,coefficient_transmission_thermique_paroi_non_isolee,tv003_umur_id,...,nb_baie_calc,surface_paroi_opaque_calc,surface_paroi_totale_calc_v1,surface_paroi_totale_calc_v2,qualif_surf,surface_paroi_opaque_infer,surface_paroi_opaque_deperditive_infer,b_infer,surface_paroi_opaque_exterieur_infer,td001_dpe_id
0,1048,209,TR014_001,Mur,40.27,TV001_001,NaN,0.35,NaN,NaN,...,4.0,115.057143,126.097143,126.097143,surface_paroi=surface_paroi_opaque,115.06,115.06,0.999975,115.06,291
1,1049,209,TR014_001,Mur,3.59,NaN,TV002_001,0.35,NaN,NaN,...,NaN,NaN,NaN,NaN,NONDEF,10.80,10.80,0.949735,NaN,291
2,1050,209,TR014_004,Plafond,6.49,NaN,TV002_005,0.14,NaN,NaN,...,NaN,NaN,NaN,NaN,NONDEF,49.10,49.10,0.944137,NaN,291
3,1814,364,TR014_001,Mur,31.71,TV001_001,NaN,0.38,NaN,TV003_001,...,5.0,83.447368,93.977368,93.977368,surface_paroi=surface_paroi_opaque,82.67,82.67,1.009403,82.67,493
4,1815,364,TR014_001,Mur,3.89,NaN,TV002_001,0.32,NaN,TV003_001,...,NaN,NaN,NaN,NaN,NONDEF,12.80,12.80,0.949707,NaN,493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439933,24115581,5419822,TR014_003,Plancher,17.35,TV001_261,NaN,0.25,NaN,NaN,...,NaN,69.400000,NaN,NaN,surface_paroi=surface_paroi_opaque,69.40,69.40,1.000000,NaN,9431410
439934,24115582,5419822,TR014_001,Mur,45.94,TV001_001,NaN,0.40,NaN,TV003_051,...,10.0,114.850000,133.490000,133.490000,surface_paroi=surface_paroi_opaque,114.86,114.86,0.999913,114.86,9431410
439935,24115588,5419822,TR014_001,Mur,5.60,NaN,TV002_001,0.40,NaN,TV003_051,...,NaN,NaN,NaN,NaN,NONDEF,17.50,17.50,0.800000,NaN,9431410
439936,24115589,5419822,TR014_004,Plafond,9.33,TV001_001,NaN,0.23,NaN,NaN,...,NaN,40.565217,NaN,NaN,surface_paroi=surface_paroi_opaque,40.55,40.55,1.000375,40.55,9431410


In [40]:
is_mext_isole

,is_murs_ext_isole
td001_dpe_id,
999890,True
999768,True
99972,True
999298,True
999222,True
...,...
1000866,True
1000817,True
1000437,True
